## 例6.6.1 一个最简单的CGE模型

In [200]:
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Alias, Domain,Sense
from gamspy.math import log
import pandas as pd

### Data


In [201]:
data = pd.read_excel("ch6-6-1.xlsx", index_col=0)
data

,sec1,sec2,lab,hh,total
sec1,4.0,3.0,NaN,3.0,10.0
sec2,2.0,5.0,NaN,6.0,13.0
lab,4.0,5.0,NaN,NaN,9.0
hh,NaN,NaN,9.0,NaN,9.0
total,10.0,13.0,9.0,9.0,NaN


### Sets

In [202]:
m = Container()
ac = Set(m, name="ac", description="all units", records=data.index.values)
acp = Alias(m, name="acp", alias_with=ac)
i = Set(m, name="i", domain=[ac], records=data.index[:2].values)
j = Alias(m, name="j", alias_with=i)

### Parameters

In [203]:
sam = Parameter(m, name="sam", description="original sam table", domain=[ac,acp], records=data.stack().reset_index())

p0 = Parameter(m, name="p0", description="inital price of commodities", domain=[i])
p0[i] = 1

w0 = Parameter(m, name="w0", description="inital price of factors")
w0[...] = 1

q0 = Parameter(m, name="q0", description="initial real total output", domain=[i])
q0[i] = sam[i,"total"]/p0[i]

x0 = Parameter(m, name="x0", description="intial real factor input", domain=[i])
x0[i] = sam["lab",i]/w0

xe0 = Parameter(m, name="e0", description="real factor endowments")
xe0[...] = sam["lab", "total"]/w0

Y0 = Parameter(m, name="Y0", description="intial income")
Y0[...] = xe0 * w0

a = Parameter(m, name="a", description="input-output coefficients", domain=[i,j])
a[i,j] = ( sam[i,j]/p0[i] ) / (q0[j]/p0[j])

b = Parameter(m, name="b", description="factor input coefficients", domain=[i])
b[i] = (sam["lab",i]/w0) / (q0[i]/p0[i])

h0 = Parameter(m, name="h0", description="initial household demand", domain=[i])
h0[i] = sam[i,"hh"]/p0[i]

alpha = Parameter(m, name="alpha", description="cobb-douglas demand function parameter", domain=[i])
alpha[i] = sam[i,"hh"]/sam["total","hh"] 



### Variables

In [204]:
p = Variable(m, name="p", description="commodities price", domain=[i])
q = Variable(m, name="q", description="commodities quantities", domain=[i])

w = Variable(m, name="w", description="factor price", )
x = Variable(m ,name="x", description="factor quantities",domain=[i])

# xe = Variable(m, name="xe", description="endowment")
Y = Variable(m, name="Y", description="household income")
h = Variable(m, name="h", description="house demand", domain=[i])

# obj = Variable(m, name="obj", description="dummy variable for solution")  


### Equations

In [205]:
# 厂商生产
production = Equation(m, name="production", description="firm production equation", domain=[i])
production[i] = p[i] == Sum(j,a[j,i]*p[j]) + w * b[i] 

# 厂商要素需求
factor = Equation(m, name="factor", description="factor demand equation", domain=[i])
factor[i] = x[i] == b[i]*q[i] 

# 家户收入
income = Equation(m, name="income")
income[...] = Y == w * xe0 

# 家户需求
demand = Equation(m, name="demand", description="household deman", domain=[i])
demand[i] = h[i] == alpha[i] * Y / p[i]

# 产品市场均衡
cmarket = Equation(m, name="cmarket", description="commodities market balance", domain=[i])
cmarket[i] = q[i] == Sum(j, a[i,j]*q[j]) + h[i] 

# 要素市场均衡
fmarket = Equation(m, name="fmarket", description="factor market balance")
fmarket[...] =  Sum(i,x[i]) == xe0

# dummy = Equation(m, "dummy")
# dummy[...] = obj == 0


### Model

In [206]:
#cge = Model(m,name="cge", problem="LP",equations=m.getEquations(),sense="min", objective=obj)
# xe.fx[...] = xe0
p.l[i] = p0[i]
q.l[i] = q0[i]
w.l[...] = w0
x.l[i] = x0[i]
h.l[i] = h0[i]
Y.l[...] = Y0
cge = Model(m, name="cge", problem="mcp", matches={production:p,factor:x,income:Y,demand:h,cmarket:q,fmarket:w})
cge.solve()  

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,2.77555756156289E-16,10,10,MCP,PATH,0.005


In [207]:
p.records

,i,level,marginal,lower,upper,scale
0,sec1,1.0,-5.551115e-17,-inf,inf,1.0
1,sec2,1.0,0.000000e+00,-inf,inf,1.0


In [208]:
h.records

,i,level,marginal,lower,upper,scale
0,sec1,3.0,0.0,-inf,inf,1.0
1,sec2,6.0,0.0,-inf,inf,1.0


In [209]:
Y.records

,level,marginal,lower,upper,scale
0,9.0,0.0,-inf,inf,1.0


In [210]:
q.records

,i,level,marginal,lower,upper,scale
0,sec1,10.0,0.0,-inf,inf,1.0
1,sec2,13.0,0.0,-inf,inf,1.0


In [211]:
cge = Model(m, name="cge", problem="mcp", matches={production:p,factor:x,income:Y,demand:h,cmarket:q,fmarket:w})
# xe.fx[...] = xe0
xe0[...] = 1.1 * sam["lab", "total"]/w0
p.l[i] = p0[i]
q.l[i] = q0[i]
w.l[...] = w0
x.l[i] = x0[i]
h.l[i] = h0[i]
Y.l[...] = Y0
cge.solve()  

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,1.77635683940025E-15,10,10,MCP,PATH,0.008


In [212]:
q.records

,i,level,marginal,lower,upper,scale
0,sec1,11.0,-8.881784e-16,-inf,inf,1.0
1,sec2,14.3,0.000000e+00,-inf,inf,1.0


In [213]:
h.records

,i,level,marginal,lower,upper,scale
0,sec1,3.3,4.440892e-16,-inf,inf,1.0
1,sec2,6.6,8.881784e-16,-inf,inf,1.0
